In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

In [2]:

# Supply capacities at each source
supply = {'Harare': 100, 'Bulawayo': 150, 'Mutare': 120}
supply_values = list(supply.values())
num_sources = len(supply)

# Demand at each destination
demand = {'Gweru': 80, 'Masvingo': 170, 'Kwekwe': 120}
demand_values = list(demand.values())
num_destinations = len(demand)

# Cost of transporting one unit from each source to each destination (Zimbabwean Dollars - ZWL)
costs_data = [
    [2, 3, 4],   # Harare to Gweru, Masvingo, Kwekwe
    [5, 1, 2],   # Bulawayo to Gweru, Masvingo, Kwekwe
    [3, 4, 1]    # Mutare to Gweru, Masvingo, Kwekwe
]
costs_df = pd.DataFrame(costs_data, index=supply.keys(), columns=demand.keys())
cost_list = costs_df.values.flatten()

# We now create the constraint matrices and vectors for linprog

# Supply constraints (equality constraints since total supply equals total demand in this balanced problem)
A_eq_supply = np.zeros((num_sources, num_sources * num_destinations))
for i in range(num_sources):
    A_eq_supply[i, i * num_destinations : (i + 1) * num_destinations] = 1
b_eq_supply = supply_values

# Demand constraints (equality constraints)
A_eq_demand = np.zeros((num_destinations, num_sources * num_destinations))
for j in range(num_destinations):
    for i in range(num_sources):
        A_eq_demand[j, i * num_destinations + j] = 1
b_eq_demand = demand_values

# Combine equality constraints
A_eq = np.vstack((A_eq_supply, A_eq_demand))
b_eq = np.hstack((b_eq_supply, b_eq_demand))

# Bounds for decision variables (non-negative)
bounds = [(0, None)] * (num_sources * num_destinations)

#  Solving the Linear Programming Problem ---
result = linprog(cost_list, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')

#  Analyzing and presenting the results
if result.success:
    print("Optimal solution found:")
    optimal_flow = np.reshape(result.x, (num_sources, num_destinations))
    optimal_df = pd.DataFrame(optimal_flow, index=supply.keys(), columns=demand.keys())
    print("\nOptimal Transportation Plan (Units transported from Source to Destination):\n")
    print(optimal_df)
    print(f"\nTotal Minimum Transportation Cost: ZWL ${result.fun:.2f}")
else:
    print("Optimal solution not found.")
    print(f"Reason: {result.message}")

Optimal solution found:

Optimal Transportation Plan (Units transported from Source to Destination):

          Gweru  Masvingo  Kwekwe
Harare     80.0      20.0     0.0
Bulawayo    0.0     150.0     0.0
Mutare      0.0       0.0   120.0

Total Minimum Transportation Cost: ZWL $490.00


C:\Users\rusha\AppData\Local\Temp\ipykernel_10040\970135612.py:50: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result = linprog(cost_list, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
C:\Users\rusha\AppData\Local\Temp\ipykernel_10040\970135612.py:50: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  result = linprog(cost_list, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
